## Import Packages

In [1]:
using ReactiveMP, Rocket, Random, GraphPPL, Distributions, LinearAlgebra, SpecialFunctions
using BenchmarkTools

## Settings

In [2]:
# seed for reproducibility
rng = MersenneTwister(1234)

# model parameters
θ_true = 0.7
α = 1.0
β = 1.5  

# benchmark settings
BenchmarkTools.DEFAULT_PARAMETERS.seconds = 60;

## Generate data

In [3]:
generate_data(rng, θ; nr_samples=10) = rand(rng, Bernoulli(θ), nr_samples) |> bool2float

function bool2float(x::Vector{Bool})
    n = length(x)
    y = Vector{Float64}(undef,n)
    for i=1:n
        x[i] ? y[i] = 1.0 : y[i] = 0.0
    end
    return y
end;

## ReactiveMP.jl scale factor extension

In [4]:
#create rules (both message and scalefactor) for Bernoulli node
@rule Bernoulli(:p, Marginalisation) (m_out::PointMass, meta::ScaleFactorMeta, ) = begin 
    r = mean(m_out)
    message = Beta(one(r) + r, 2one(r) - r)
    scalefactor = -log(0.5)
    return ScaledMessage(message, scalefactor)
end

#create rules (both message and scalefactor) for Beta node
@rule Beta(:out, Marginalisation) (m_a::PointMass, m_b::PointMass, meta::ScaleFactorMeta) = begin
    message = @call_rule Beta(:out, Marginalisation) (m_a = m_a, m_b = m_b)
    scalefactor = 0.0
    return ScaledMessage(message, scalefactor)
end

In [5]:
#Product function for equality node
function ReactiveMP.prod(::ProdAnalytical, left::ScaledMessage{ <: Beta }, right::ScaledMessage{ <: Beta })
    a_left, b_left  = Distributions.params(left.message)
    a_right, b_right = Distributions.params(right.message)

    message = prod(ProdAnalytical(),left.message,right.message)
    scalefactor = left.scale + right.scale - log(beta(a_left + a_right - 1, b_left + b_right - 1)) +
                    log(beta(a_left, b_left)) + log(beta(a_right, b_right))

    return ScaledMessage(message,scalefactor)
end

## Inference by ReactiveMP (scale factors)

In [6]:
@model [ default_meta = ScaleFactorMeta() ] function model_cointoss_scalefactor(α, β; nr_samples=10)

    θ ~ Beta(α, β)

    y = datavar(Float64, nr_samples)

    for n = 1:nr_samples
        y[n] ~ Bernoulli(θ)
    end

    return y, θ
end

model_cointoss_scalefactor (generic function with 1 method)

In [7]:
function inference_cointoss_scalefactor(data, α, β; nr_samples=10)

    model, (y, θ) = model_cointoss_scalefactor(α, β; nr_samples = nr_samples, options = (limit_stack_depth = 500, ));

    θ_mar_sf = keep(Marginal)

    θ_sub = subscribe!(getmarginal(θ), θ_mar_sf)

    bmark_sf = @benchmark update!($y, $data)

    unsubscribe!(θ_sub)

    return θ_mar_sf, bmark_sf
end

inference_cointoss_scalefactor (generic function with 1 method)

## Inference by ReactiveMP (Bethe free energy)

In [8]:
@model function model_cointoss_bfe(α, β; nr_samples=10)

    θ ~ Beta(α, β)

    y = datavar(Float64, nr_samples)

    for n = 1:nr_samples
        y[n] ~ Bernoulli(θ)
    end
    
    return y, θ
end

model_cointoss_bfe (generic function with 1 method)

In [9]:
function inference_cointoss_bfe(data, α, β; nr_samples=10)

    model, (y, θ) = model_cointoss_bfe(α, β; nr_samples = nr_samples, options = (limit_stack_depth = 500, ));

    θ_mar = keep(Marginal)
    bfe = keep(Float64)

    θ_subscribe = subscribe!(getmarginal(θ), θ_mar);

    bfe_subscribe = subscribe!(score(Float64,BetheFreeEnergy(), model), bfe);

    bmark = @benchmark update!($y, $data)
    
    unsubscribe!((θ_subscribe, bfe_subscribe))

    return θ_mar, bfe, bmark
end

inference_cointoss_bfe (generic function with 1 method)

## Correctness check and performance comparison

#### Scale factors (N=10)

In [10]:
data = generate_data(rng, θ_true; nr_samples=10);

In [11]:
θ_mar_sf, bmark_sf = inference_cointoss_scalefactor(data, α, β; nr_samples=10);
println(-θ_mar_sf[end].data.scale)
bmark_sf

-7.349984471920427


BenchmarkTools.Trial: 10000 samples with 5 evaluations.
 Range (min … max):  6.400 μs …  3.054 ms  ┊ GC (min … max):  0.00% … 99.29%
 Time  (median):     6.960 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   8.775 μs ± 58.925 μs  ┊ GC (mean ± σ):  13.37% ±  1.99%

   ▄█▆                                                        
  ▃███▇▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▃▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  6.4 μs         Histogram: frequency by time        13.9 μs <

 Memory estimate: 6.12 KiB, allocs estimate: 114.

#### Bethe free energy (N=10)

In [12]:
θ_mar_bfe, bfe, bmark_bfe = inference_cointoss_bfe(data, α, β; nr_samples=10);
println(-bfe[end])
bmark_bfe

-7.349984471920429


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  25.300 μs …  26.079 ms  ┊ GC (min … max):  0.00% … 99.54%
 Time  (median):     28.300 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   35.594 μs ± 365.255 μs  ┊ GC (mean ± σ):  14.47% ±  1.41%

   █▇▄▂                                                         
  ▇█████▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  25.3 μs         Histogram: frequency by time         75.4 μs <

 Memory estimate: 22.33 KiB, allocs estimate: 512.

#### Scale factors (N=100)

In [13]:
data = generate_data(rng, θ_true; nr_samples=100);

In [14]:
θ_mar_sf, bmark_sf = inference_cointoss_scalefactor(data, α, β; nr_samples=100);
println(-θ_mar_sf[end].data.scale)
bmark_sf

-57.64998869192096


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  64.600 μs …  26.146 ms  ┊ GC (min … max):  0.00% … 99.45%
 Time  (median):     67.900 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   89.448 μs ± 621.685 μs  ┊ GC (mean ± σ):  16.98% ±  2.44%

  ▇█▆▄▃▃▃▃▄▅▅▄▃▃▂▁▁                                            ▂
  ████████████████████▇▇▆▆▇▆▇▆█▇▆▆▅▅▆▄▆▆▆▅▄▄▅▆▅▅▃▄▄▅▅▄▅▄▄▄▄▃▃▅ █
  64.6 μs       Histogram: log(frequency) by time       155 μs <

 Memory estimate: 59.56 KiB, allocs estimate: 1104.

#### Bethe free energy (N=100)

In [15]:
θ_mar_bfe, bfe, bmark_bfe = inference_cointoss_bfe(data, α, β; nr_samples=100);
println(-bfe[end])
bmark_bfe

-57.64998869192087


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  234.500 μs … 22.721 ms  ┊ GC (min … max):  0.00% … 98.22%
 Time  (median):     244.700 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   307.073 μs ±  1.029 ms  ┊ GC (mean ± σ):  15.95% ±  4.70%

  ▇█▇▇▅▄▄▄▃▂▂▁▂▁▁                                              ▂
  █████████████████████▇▇▇▇▇▆▇▅▆▇▇▇▇▇▆▇▇▇▇▆▇▇▆▄▅▄▅▄▄▅▄▄▄▄▃▃▂▄▄ █
  234 μs        Histogram: log(frequency) by time       473 μs <

 Memory estimate: 210.77 KiB, allocs estimate: 4832.

#### Scale factors (N=1000)

In [16]:
data = generate_data(rng, θ_true; nr_samples=1000);

In [17]:
θ_mar_sf, bmark_sf = inference_cointoss_scalefactor(data, α, β; nr_samples=1000);
println(-θ_mar_sf[end].data.scale)
bmark_sf

-616.8891152516392


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  641.700 μs … 20.715 ms  ┊ GC (min … max):  0.00% … 92.29%
 Time  (median):     699.050 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   877.647 μs ±  1.099 ms  ┊ GC (mean ± σ):  10.33% ±  7.94%

  ██▆▅▅▅▅▄▄▄▃▃▂▂▂▁▁▁▁▁                                         ▂
  █████████████████████████▇▇▆▆▇▆▅▆▅▆▅▆▇▆▅▅▅▆▆▅▄▄▅▃▄▄▅▄▄▄▄▄▃▁▄ █
  642 μs        Histogram: log(frequency) by time      2.43 ms <

 Memory estimate: 593.94 KiB, allocs estimate: 11004.

#### Bethe free energy (N=1000)

In [18]:
θ_mar_bfe, bfe, bmark_bfe = inference_cointoss_bfe(data, α, β; nr_samples=1000);
println(-bfe[end])
bmark_bfe

-616.8891152516348


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  2.614 ms … 18.681 ms  ┊ GC (min … max): 0.00% … 76.50%
 Time  (median):     3.779 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.211 ms ±  2.237 ms  ┊ GC (mean ± σ):  8.73% ± 12.80%

   ▆██▇▆▅▁▃                                                   
  ▅█████████▅▃▂▂▂▂▂▁▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃ ▃
  2.61 ms        Histogram: frequency by time        16.5 ms <

 Memory estimate: 2.06 MiB, allocs estimate: 49046.